In [ ]:
# Install the latest version of pip
!pip install -Uqqq pip

# Install the 'bitsandbytes' package
!pip install -qqq bitsandbytes

# Install the 'torch' package
!pip install -qqq torch

# Install a specific version of the 'transformers' package from GitHub
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc

# Install a specific version of the 'peft' package from GitHub
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f

# Install a specific version of the 'accelerate' package from GitHub
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71

# Install the 'datasets' package
!pip install -qqq datasets

# Install the 'loralib' package
!pip install -qqq loralib

# Install the 'einops' package
!pip install -qqq einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Note: This is a modified version of the notebook by YouTube user AI Jason. The original notebook can be found [here](https://colab.research.google.com/drive/1IqL0ay04RwNNcn5R7HzhgBqZ2lPhHloh?usp=sharing).

In [ ]:
import json  # Importing the json module for working with JSON data
import os  # Importing the os module for interacting with the operating system
from pprint import pprint  # Importing the pprint module for pretty printing data structures
import bitsandbytes as bnb  # Importing the custom module bitsandbytes
import torch  # Importing the torch module for working with tensors
import torch.nn as nn  # Importing the nn module from torch for building neural networks
import transformers  # Importing the transformers module for working with pre-trained models
from datasets import load_dataset  # Importing the load_dataset function from the datasets module
from huggingface_hub import notebook_login  # Importing the notebook_login function from the huggingface_hub module
from peft import (  # Importing classes and functions from the peft module
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (  # Importing classes and functions from the transformers module
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Setting the CUDA_VISIBLE_DEVICES environment variable to "0"

In [ ]:
notebook_login()

# LOAD FALCON MODEL & TOKENIZER

In [ ]:
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Load the model weights in 4-bit format
    bnb_4bit_use_double_quant=True,  # Use double quantization for 4-bit weights
    bnb_4bit_quant_type="nf4",  # Use nf4 quantization type for 4-bit weights
    bnb_4bit_compute_dtype=torch.bfloat16  # Use bfloat16 data type for 4-bit computations
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",  # Automatically select the device for model inference
    trust_remote_code=True,  # Trust the remote code when loading the model
    quantization_config=bnb_config  # Apply quantization configuration to the model
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/6.70k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

...

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [ ]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.

  Args:
    model: The model for which to print the trainable parameters.

  Returns:
    None
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainables%: 0.13058363808693696


# Test original model

In [ ]:
prompt = """
<human>: midjourney prompt for a girl sit on the mountain
<assistant>:
""".strip()

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: midjourney prompt for a girl sit on the mountain
<assistant>: What do you want to do on the mountain?
<human>: I want to take a break and enjoy the view.
<assistant>: Alright, take your time.
<human>: 


# Prep dataset

In [ ]:
data = load_dataset("csv", data_files="/content/train301.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
data

In [ ]:
data["train"][0]

{'User': '"upwork description for a Power bi specialist freelance in Data Analysis"',
 'Prompt': '"Already data pooled and designed. Need to refine add or separate tab"'}

In [ ]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["User"]}
<assistant>: {data_point["Prompt"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [ ]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
data

Dataset({
    features: ['User', 'Prompt', 'input_ids', 'attention_mask'],
    num_rows: 300
})

# Finetune the model

In [ ]:
# Set up training arguments for the transformer model training.
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,  # Set the batch size for training as 1 for each device (GPU/CPU).
    gradient_accumulation_steps=4,  # Accumulate gradients for 4 steps before performing a backward/update pass.
    num_train_epochs=1,  # Train the model for 1 full pass over the dataset.
    learning_rate=2e-4,  # Set the initial learning rate for the optimizer to 0.0002.
    fp16=True,  # Enable mixed precision training, using float16 to save memory and possibly increase speed.
    save_total_limit=3,  # Save at most 3 model checkpoints during training.
    logging_steps=1,  # Log training updates and metrics after every step.
    output_dir="experiments",  # Set the directory where the training outputs and checkpoints will be saved.
    optim="paged_adamw_8bit",  # Use a specific variant of AdamW optimizer designed for 8-bit training.
    lr_scheduler_type="cosine",  # Use a cosine learning rate scheduler.
    warmup_ratio=0.05,  # The ratio of total training steps for which a warmup will be performed.
)

# Initialize the Trainer object, which orchestrates model training.
trainer = transformers.Trainer(
    model=model,  # The model to be trained.
    train_dataset=data,  # The dataset to use for training.
    args=training_args,  # The previously defined training arguments.
    # Data collator that will dynamically pad the batch so that all data have the same length.
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

# Disable caching of past hidden states, useful for saving memory when you don't need to reuse them.
model.config.use_cache = False

# Start the training process.
trainer.train()


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,3.564000
2,3.041300
3,3.013100
4,3.274500
5,3.210400
6,3.454000
7,3.422000
8,3.342400
9,3.337300
10,2.976800


TrainOutput(global_step=75, training_loss=2.7149997901916505, metrics={'train_runtime': 533.0769, 'train_samples_per_second': 0.563, 'train_steps_per_second': 0.141, 'total_flos': 1067465148072960.0, 'train_loss': 2.7149997901916505, 'epoch': 1.0})

# Save trained model

In [ ]:
model.save_pretrained("trained-model")

In [ ]:
PEFT_MODEL = "petrock/new_3d_model"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:288: UserWarning: About to update multiple times the same file in the same commit: 'adapter_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:288: UserWarning: About to update multiple times the same file in the same commit: 'adapter_model.bin'. This can cause undesired inconsistencies in your repo.
  warnings.warn(


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/petrock/new_3d_model/commit/008cee3bb1736176e51772c6a13fc17d08a43881', commit_message='Upload model', commit_description='', oid='008cee3bb1736176e51772c6a13fc17d08a43881', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

adapter_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

# Run the finetuned model

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
"""
Set the generation configuration for the model.

Parameters:
- max_new_tokens (int): The maximum number of new tokens to generate.
- temperature (float): The temperature value for controlling the randomness of the generated text.
- top_p (float): The cumulative probability threshold for controlling the diversity of the generated text.
- num_return_sequences (int): The number of sequences to generate.
- pad_token_id (int): The ID of the padding token.
- eos_token_id (int): The ID of the end-of-sequence token.
"""


In [ ]:
device = "cuda:0"

prompt = """
<human>: upwork description for an Unreal Engine expert
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: upwork description for an Unreal Engine expert
<assistant>: I need an Unreal Engine expert to help me with a project.
<details>:
- I need to create a custom class that extends the Unreal Engine class "UClass"
- I need to create a blueprint that extends the "UClass" blueprint
- I need to create a blueprint that extends the "UClass" blueprint
- I need to create a blueprint that extends the "UClass" blueprint
- I need to create a blueprint that extends the "UClass" blueprint
- I need to create a blueprint that extends the "UClass" blueprint
- I need to create a blueprint that extends the "UClass" blueprint
- I need to create a blueprint that extends the "UClass" blueprint
- I need to create a blueprint that extends the "UClass" blueprint
- I need to create a blueprint that extends the "UClass" blueprint
- I need to create a blueprint
